In [1]:
# modules

import numpy as np
from numpy.lib.function_base import average
import pandas as pd
from pathlib import Path
import os
from sklearn.metrics import f1_score, accuracy_score
import argparse

In [7]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--version', type=str, default='041')
    parser.add_argument('--gpu', type=int, default=0)
    return parser.parse_args([])

In [9]:
%cd ../

/home/hirano/work/Competition/Nishika_cable/src


In [24]:
args = get_args()
version = args.version
folds = [0,1,2,3,4,5,6,7,8,9]

In [27]:
sub = pd.read_csv('../input/test_with_fold.csv')
pred = np.zeros((len(sub), 2))

train = pd.read_csv('../input/train_with_fold_USBmicro.csv')
target = train.target.values
oof = pd.DataFrame()
oof_cls = pd.DataFrame()

for fold in folds:
    df = pd.read_csv(f'../output/{version}/{fold}/sub_{version}_raw.csv')
    pred += df.values
    oof_raw_name = [x for x in os.listdir(f'../output/{version}/{fold}/') if ('oof' in x)&('raw' in x)]
    oof_cls_name = [x for x in os.listdir(f'../output/{version}/{fold}/') if ('oof' in x)&('raw' not in x)]

    print(f"Fold is {fold}", oof_raw_name, oof_cls_name)

    _oof = pd.read_csv(f'../output/{version}/{fold}/{oof_raw_name[0]}')
    oof = pd.concat([oof, _oof], axis=0)

    _oof = pd.read_csv(f'../output/{version}/{fold}/{oof_cls_name[0]}')
    oof_cls = pd.concat([oof_cls, _oof], axis=0)
    
oof.columns = ['conf_0', 'conf_1']

_oof = oof.copy()

_oof['pred'] = np.argmax(_oof.values, axis=1)

oof_pred = _oof.pred.values
oof_target = oof_cls.target

Fold is 0 ['oof_041_0.9802_raw.csv'] ['oof_041_0.9802.csv']
Fold is 1 ['oof_041_0.9901_raw.csv'] ['oof_041_0.9901.csv']
Fold is 2 ['oof_041_1.0000_raw.csv'] ['oof_041_1.0000.csv']
Fold is 3 ['oof_041_1.0000_raw.csv'] ['oof_041_1.0000.csv']
Fold is 4 ['oof_041_0.9900_raw.csv'] ['oof_041_0.9900.csv']
Fold is 5 ['oof_041_0.9900_raw.csv'] ['oof_041_0.9900.csv']
Fold is 6 ['oof_041_0.9800_raw.csv'] ['oof_041_0.9800.csv']
Fold is 7 ['oof_041_0.9800_raw.csv'] ['oof_041_0.9800.csv']
Fold is 8 ['oof_041_1.0000_raw.csv'] ['oof_041_1.0000.csv']
Fold is 9 ['oof_041_0.9900_raw.csv'] ['oof_041_0.9900.csv']


In [28]:
score = f1_score(oof_target, oof_pred, average='micro')
print(score)

0.9900199600798403


In [29]:
sub.target = np.argmax(pred, axis=1)

In [31]:
oof_cls.to_csv(f'../output/post_process/oof_{version}_{score:.4f}.csv', index=False)

# Test file

In [37]:
pred /= len(folds)

In [38]:
pred

array([[0.99365234, 0.00635462],
       [0.99575195, 0.00421529],
       [0.99072266, 0.00922441],
       ...,
       [0.99052734, 0.00946276],
       [0.99418945, 0.00576289],
       [0.97646484, 0.0235817 ]])

In [40]:
sub[['conf_0', 'conf_1']] = pred

In [43]:
sub.target = np.argmax(pred, axis=1)

In [45]:
sub[['target','conf_0', 'conf_1']].to_csv(f'../output/post_process/sub_{version}.csv', index=False)